In [1]:
!pip install -q snscrape==0.3.4


In [2]:
import os
import pandas as pd
from datetime import date
import re
import string

In [3]:
today = date.today()
end_date = today

##Search List





In [4]:
search_term = ['aap','bsp','bjp','sp','congress']
from_date = '2022-02-24'

##Find the number of tweets

In [5]:
for i in search_term:
  os.system(f"snscrape --since {from_date} twitter-search '{i} until:{end_date}' >{i}.txt")
  if os.stat(f"{i}.txt").st_size == 0:
    counter = 0
  else:
    df = pd.read_csv(f'{i}.txt', names=['link'])
    counter = df.size

  print(f'Number Of Tweets {i}: '+ str(counter))

Number Of Tweets aap: 668
Number Of Tweets bsp: 743
Number Of Tweets bjp: 669
Number Of Tweets sp: 331
Number Of Tweets congress: 200


In [6]:
max_results = 100


##Extract Data from tweets


In [7]:
for i in search_term:
  extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{i} until:{end_date}' > extracted-tweets{i}.txt"
  os.system(extracted_tweets)
  if os.stat(f"extracted-tweets{i}.txt").st_size == 0:
    print('No Tweets found')
  else:
    df = pd.read_csv(f'extracted-tweets{i}.txt', names=['content'])
    for row in df['content'].iteritems():
      print(row)

(0, "'@Rizwank3315 Bhai aap ko dm karna he'")
(1, "'@teamimrankhanpm Bismillah Hir Rahma Nir Rahim MashaAllah MashaAllah MashaAllah Aap Ki Team Bahot Behtareen R Khoobsurat Ho Gi.'")
(2, "'@Shailes47245787 @Alienkumar1 @TeamKangana5 Bhai aap jaise logo ko sirf kamiya hi dikhti hai ")
(3, "'@mehmoodkakar604 @ShakilRai2 @YouTube AUS SE BARRA NAWAZ AKA BAKWAAS SHARIF FUNDING CHOR. CHOR KO IZAT DO. AAP BHI DO.'")
(4, "'@ShubhiM42016011 Aap aaj aayiye thik hai'")
(5, "'@ajaydevgn @DisneyPlusHS sir vimal khana kab choroge aap'")
(6, "'Kitne hurdles aae lekin apna kaam karte rahe. And apne kaam se kaam rakha. No hype no drama no loud claims. Just working silently and determinedly. Aap log thak kar beech mein stop bhi kar dete to kisi ko pata nahin chalta. What drives You Shah. May the movie get completed fast'")
(7, "Aap yadi kisi aesi cheez par kabja karke baithe ho. IN GENERAL SENSE Let's say ki woh satta ki kursi hi hai.\\n\\nPar yadi woh cheez aapke jiwan ko narak bana rahi hai.\\n\\nFir 

####All extracted files merge into one

In [8]:
filenames = ["extracted-tweetsaap.txt", "extracted-tweetsbjp.txt", "extracted-tweetsbsp.txt","extracted-tweetscongress.txt","extracted-tweetssp.txt"]

with open("output_file.txt", "w") as outfile:
    for filename in filenames:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)


###Converted into CSV with column name

In [9]:
df = pd.read_csv('output_file.txt', sep='delimiter', header=None, engine='python', index_col=False,names=['text'])
df.to_csv('output_file.csv')
df.head()

,text
0,'@Rizwank3315 Bhai aap ko dm karna he'
1,'@teamimrankhanpm Bismillah Hir Rahma Nir Rahi...
2,'@Shailes47245787 @Alienkumar1 @TeamKangana5 B...
3,'@mehmoodkakar604 @ShakilRai2 @YouTube AUS SE ...
4,'@ShubhiM42016011 Aap aaj aayiye thik hai'


#Cleaning Data

###Remove username & punctuations

In [10]:
def remove_username_no_punctuation(test_str):
    test_str = re.sub('@[^\s]+','',test_str)
    for char in test_str:
        if char in string.punctuation:
            test_str = test_str.replace(char, "")
    return test_str

In [11]:
df['text'] = df['text'].apply(remove_username_no_punctuation)
df.head()

,text
0,Bhai aap ko dm karna he
1,Bismillah Hir Rahma Nir Rahim MashaAllah Mash...
2,Bhai aap jaise logo ko sirf kamiya hi dikht...
3,AUS SE BARRA NAWAZ AKA BAKWAAS SHARIF FUNDI...
4,Aap aaj aayiye thik hai


###Remove Emojis

In [12]:
def remove_emoji(test_str):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"  
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',test_str)

In [13]:
df['text'] = df['text'].apply(remove_emoji)
df.head()

,text
0,Bhai aap ko dm karna he
1,Bismillah Hir Rahma Nir Rahim MashaAllah Mash...
2,Bhai aap jaise logo ko sirf kamiya hi dikht...
3,AUS SE BARRA NAWAZ AKA BAKWAAS SHARIF FUNDI...
4,Aap aaj aayiye thik hai


###Find the null values

In [14]:
df.isnull().sum()

text    0
dtype: int64

###Labeling the data in search term data

In [15]:
def labeling(new_string):
    for char in new_string.lower().split():
        if char in search_term:
            return 1
    else:
        return 0

In [17]:
df['Label'] = df['text'].apply(labeling)
df.to_csv('output_file.csv')

###Count the values 

In [18]:
df['Label'].value_counts()


1    277
0    223
Name: Label, dtype: int64

In [19]:
df.to_csv('output_file.csv')